## Initialization  

In [7]:
%load_ext autoreload
%autoreload 2
import os 
import sys
sys.path.insert(0, '../src')
import time
import argparse
import yaml
import types
import copy, pprint
from time import sleep
from datetime import datetime
import numpy  as np
import torch  
import wandb
import pandas as pd
from utils.notebook_modules import initialize, init_dataloaders, init_environment, init_wandb, \
                                   training_prep, disp_dataloader_info,disp_info_1, \
                                   warmup_phase, weight_policy_training, disp_gpu_info

from utils import print_separator, print_heading, timestring, print_loss, load_from_pickle

pp = pprint.PrettyPrinter(indent=4)
np.set_printoptions(edgeitems=3, infstr='inf', linewidth=150, nanstr='nan')
torch.set_printoptions(precision=6, linewidth=132)
pd.options.display.width = 132
# torch.set_printoptions(precision=None, threshold=None, edgeitems=None, linewidth=None, profile=None, sci_mode=None)
# sys.path.insert(0, '/home/kbardool/kusanagi/AdaSparseChem/src')
# print(sys.path)
# disp_gpu_info() 
os.environ["WANDB_NOTEBOOK_NAME"] = "Adashare_Training.ipynb"


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Create Environment

### Parse Input Args  - Read YAML config file

In [8]:
# RESUME_MODEL_CKPT = 'model_train_ep_25_seed_0088'

## For RESTARTING
##
# input_args = " --config yamls/chembl_3task_train.yaml " \
#              " --resume " \
#              " --exp_id      330i85cg" \
#              " --exp_name    0308_1204" \
#              " --exp_desc    Train with dropout 0.5" \
#              " --seed_idx    0 "\
#              " --batch_size  128" \
#              " --lambda_sparsity  0.01"\
#              " --lambda_sharing   0.01" 
## get command line arguments

In [9]:
##  For Initiating 
##
input_args = " --config ../yamls/chembl_synt_train.yaml " \
             " --exp_desc            dropout 0.5, weight 105 bch/ep policy 105 bch/ep " \
             " --warmup_epochs       150 " \
             " --hidden_size         40 40 40 40 " \
             " --tail_hidden_size    40" \
             " --seed_idx             0" \
             " --batch_size         128" \
             " --task_lr          0.001" \
             " --backbone_lr      0.001" \
             " --policy_lr        0.001" \
             " --lambda_sparsity   0.02" \
             " --lambda_sharing    0.01" \
               " --folder_sfx    noplcy"                       
#              " --hidden_size   100 100 100 100 100 100" \
#              " --tail_hidden_size  100 " \


In [10]:
opt, ns = initialize(input_args, build_folders = True)


  command line parms : 
------------------------
 config...................  ../yamls/chembl_synt_train.yaml
 exp_id...................  None
 exp_name.................  None
 folder_sfx...............  noplcy
 exp_desc.................  dropout 0.5, weight 105 bch/ep policy 105 bch/ep
 hidden_sizes.............  [40, 40, 40, 40]
 tail_hidden_size.........  40
 warmup_epochs............  150
 training_epochs..........  None
 seed_idx.................  0
 batch_size...............  128
 backbone_lr..............  0.001
 task_lr..................  0.001
 policy_lr................  0.001
 decay_lr_rate............  None
 decay_lr_freq............  None
 lambda_sparsity..........  0.02
 lambda_sharing...........  0.01
 gpu_ids..................  [0]
 resume...................  False
 cpu......................  False



##################################################
################### READ YAML ####################
##################################################


 log_dir         

### Setup Dataloader and Model  

In [6]:
dldrs = init_dataloaders(opt, verbose = True)
disp_dataloader_info(dldrs)

environ = init_environment(ns, opt, is_train = True, policy_learning = False, display_cfg = False)

# ********************************************************************
# **************** define optimizer and schedulers *******************
# ********************************************************************                                
environ.define_optimizer(policy_learning=False)
environ.define_scheduler(policy_learning=False)


##################################################
############### CREATE DATALOADERS ###############
##################################################
----------------------------------------------------------------
 2022-04-02 19:37:19:692302  Create new  Chembl_23 instance 
---------------------------------------------------------------- 

 verbose        : True

* load y_task1 file : chembl_23mini_adashare_y1_bin_sparse.npy . . .
	 y_task[1]  All y values are 0, 1, or -1.
	 load_task_weights - filename: None label: y_task1
	 load_task_weights - no weights file provided for y_task1, training_weights for all  5 classes set to 1 
	 tasks_class.aggregation_weight WAS NOT passed 
	 min_samples_class: 5
	 Class fold counts: 
  fold_pos:
[[1382 1417 1493 1346 1435]
 [1463 1492 1350 1516 1291]
 [1498 1306 1366 1428 1333]
 [1331 1264 1205 1312 1356]
 [1375 1370 1359 1362 1339]]  

  fold_neg:
[[1363 1328 1252 1399 1310]
 [1424 1395 1537 1371 1596]
 [1129 1321 1261 1199 1294]
 [1154 1221 12

In [6]:
# environ.optimizers['weights'].param_groups[0]

###  Weights and Biases Initialization 

In [7]:
init_wandb(ns, opt, environment = environ)

print(f" PROJECT NAME: {ns.wandb_run.project}\n"
      f" RUN ID      : {ns.wandb_run.id} \n"
      f" RUN NAME    : {ns.wandb_run.name}") 

wandb: Currently logged in as: kbardool (use `wandb login --relogin` to force relogin)


1o46td06 0402_1658_noplcy AdaSparseChem


 PROJECT NAME: AdaSparseChem
 RUN ID      : 1o46td06 
 RUN NAME    : 0402_1658_noplcy
 PROJECT NAME: AdaSparseChem
 RUN ID      : 1o46td06 
 RUN NAME    : 0402_1658_noplcy


In [6]:
# ns.wandb_run.finish()

AttributeError: 'types.SimpleNamespace' object has no attribute 'wandb_run'

### Initiate / Resume Training Prep

In [9]:
if opt['train']['resume']:
    RESUME_MODEL_CKPT = ""
    RESUME_METRICS_CKPT = ""    
    print(opt['train']['which_iter'])
    print(opt['paths']['checkpoint_dir'])
    print(RESUME_MODEL_CKPT)
    # opt['train']['resume'] = True
    # opt['train']['which_iter'] = 'warmup_ep_40_seed_0088'
    print_separator('Resume training')
    loaded_iter, loaded_epoch = environ.load_checkpoint(RESUME_MODEL_CKPT, path = opt['paths']['checkpoint_dir'], verbose = True)
    print(loaded_iter, loaded_epoch)    
#     current_iter = environ.load_checkpoint(opt['train']['which_iter'])
    environ.networks['mtl-net'].reset_logits()
    val_metrics = load_from_pickle(opt['paths']['checkpoint_dir'], RESUME_METRICS_CKPT)
    # training_prep(ns, opt, environ, dldrs, epoch = loaded_epoch, iter = loaded_iter )

else:
    print_separator('Initiate Training ')

##################################################
############### Initiate Training  ###############
##################################################


### Training Preparation

In [10]:
training_prep(ns, opt, environ, dldrs)
print('-'*80)
disp_info_1(ns, opt, environ)
print('-'*80)
print(environ.disp_for_excel())

 cuda available [0]
 set print_freq to length of train loader: 105
 set eval_iters to length of val loader  : 33
--------------------------------------------------------------------------------

 Num_blocks                : 4                                

 batch size                : 128 
 batches/ Weight trn epoch : 105 
 batches/ Policy trn epoch : 105                                 

 Print Frequency           : -1 
 Config Val Frequency      : 500 
 Config Val Iterations     : -1 
 Val iterations            : 33 
 which_iter                : warmup 
 train_resume              : False                                 
 
 fix BN parms              : False 
 Task LR                   : 0.001 
 Backbone LR               : 0.001                                 

 Sharing  regularization   : 0.01 
 Sparsity regularization   : 0.02 
 Task     regularization   : 1                                 

 Current epoch             : 0  
 Warm-up epochs            : 150 
 Training epochs       

## Warmup Training

In [11]:
# environ.display_trained_policy(ns.current_epoch,out=sys.stdout)
# ns.stop_epoch_warmup = 10
ns.warmup_epochs = 2
# ns.check_for_improvment_wait = 0
print(ns.warmup_epochs, ns.current_epoch)
print_heading(f" Last Epoch: {ns.current_epoch}   # of warm-up epochs to do:  {ns.warmup_epochs} - Run epochs {ns.current_epoch+1} to {ns.current_epoch + ns.warmup_epochs}", verbose = True)

2 0
----------------------------------------------------------------------
 Last Epoch: 0   # of warm-up epochs to do:  2 - Run epochs 1 to 2
---------------------------------------------------------------------- 



In [43]:
# warmup_phase(ns,opt, environ, dldrs, epochs = 25)
warmup_phase(ns,opt, environ, dldrs)

----------------------------------------------------------------------------
 Last Epoch: 150   # of warm-up epochs to do:  2 - Run epochs 151 to 152
---------------------------------------------------------------------------- 


 + Validation Loop Start - batch_idx:1  eval_iters: 33  t_validation: 33 

 + Validation Loop - batch_idx:1  eval_iters: 33
    loss[task1]     : 0.3179    sum(err)               : 0.3179    sum(err)/batch_id      : 0.3179 
    loss_mean[task1]: 0.2949    sum(err_mean)          : 0.2949    avg(err_mean)          : 0.2949
    self.metrics[task_key][yc_wghts_sum] 138.0    task_weights[task_key] : 138.0  

 + Validation Loop end - batch_idx:1  eval_iters: 33 
    all tasks_loss_sum          : 0.3179     avg(all_tasks_loss_sum)     : 0.3179 
    all tasks_loss_sum_mean     : 0.2949    avg(all_tasks_loss_sum_mean): 0.2949

 + Validation Loop Start - batch_idx:2  eval_iters: 33  t_validation: 33 

 + Validation Loop - batch_idx:2  eval_iters: 33
    loss[task1]     

 + Validation Loop - batch_idx:17  eval_iters: 33
    loss[task1]     : 0.5093    sum(err)               : 4.8220    sum(err)/batch_id      : 0.2836 
    loss_mean[task1]: 0.4724    sum(err_mean)          : 4.4201    avg(err_mean)          : 0.2600
    self.metrics[task_key][yc_wghts_sum] 138.0    task_weights[task_key] : 2373.0  

 + Validation Loop end - batch_idx:17  eval_iters: 33 
    all tasks_loss_sum          : 4.8220     avg(all_tasks_loss_sum)     : 0.2836 
    all tasks_loss_sum_mean     : 4.4201    avg(all_tasks_loss_sum_mean): 0.2600

 + Validation Loop Start - batch_idx:18  eval_iters: 33  t_validation: 33 

 + Validation Loop - batch_idx:18  eval_iters: 33
    loss[task1]     : 0.3115    sum(err)               : 5.1335    sum(err)/batch_id      : 0.2852 
    loss_mean[task1]: 0.2712    sum(err_mean)          : 4.6913    avg(err_mean)          : 0.2606
    self.metrics[task_key][yc_wghts_sum] 147.0    task_weights[task_key] : 2520.0  

 + Validation Loop end - batch_idx:1

 + Validation Loop - batch_idx:31  eval_iters: 33
    loss[task1]     : 0.1707    sum(err)               : 8.4983    sum(err)/batch_id      : 0.2741 
    loss_mean[task1]: 0.1572    sum(err_mean)          : 7.7982    avg(err_mean)          : 0.2516
    self.metrics[task_key][yc_wghts_sum] 139.0    task_weights[task_key] : 4324.0  

 + Validation Loop end - batch_idx:31  eval_iters: 33 
    all tasks_loss_sum          : 8.4983     avg(all_tasks_loss_sum)     : 0.2741 
    all tasks_loss_sum_mean     : 7.7982    avg(all_tasks_loss_sum_mean): 0.2516

 + Validation Loop Start - batch_idx:32  eval_iters: 33  t_validation: 33 

 + Validation Loop - batch_idx:32  eval_iters: 33
    loss[task1]     : 0.2065    sum(err)               : 8.7048    sum(err)/batch_id      : 0.2720 
    loss_mean[task1]: 0.1930    sum(err_mean)          : 7.9911    avg(err_mean)          : 0.2497
    self.metrics[task_key][yc_wghts_sum] 137.0    task_weights[task_key] : 4461.0  

 + Validation Loop end - batch_idx:3

 + Validation Loop - batch_idx:11  eval_iters: 33
    loss[task1]     : 0.2600    sum(err)               : 2.7542    sum(err)/batch_id      : 0.2504 
    loss_mean[task1]: 0.2483    sum(err_mean)          : 2.5343    avg(err_mean)          : 0.2304
    self.metrics[task_key][yc_wghts_sum] 134.0    task_weights[task_key] : 1523.0  

 + Validation Loop end - batch_idx:11  eval_iters: 33 
    all tasks_loss_sum          : 2.7542     avg(all_tasks_loss_sum)     : 0.2504 
    all tasks_loss_sum_mean     : 2.5343    avg(all_tasks_loss_sum_mean): 0.2304

 + Validation Loop Start - batch_idx:12  eval_iters: 33  t_validation: 33 

 + Validation Loop - batch_idx:12  eval_iters: 33
    loss[task1]     : 0.3024    sum(err)               : 3.0566    sum(err)/batch_id      : 0.2547 
    loss_mean[task1]: 0.2707    sum(err_mean)          : 2.8050    avg(err_mean)          : 0.2337
    self.metrics[task_key][yc_wghts_sum] 143.0    task_weights[task_key] : 1666.0  

 + Validation Loop end - batch_idx:1

 + Validation Loop - batch_idx:27  eval_iters: 33
    loss[task1]     : 0.3472    sum(err)               : 7.2791    sum(err)/batch_id      : 0.2696 
    loss_mean[task1]: 0.3108    sum(err_mean)          : 6.6574    avg(err_mean)          : 0.2466
    self.metrics[task_key][yc_wghts_sum] 143.0    task_weights[task_key] : 3769.0  

 + Validation Loop end - batch_idx:27  eval_iters: 33 
    all tasks_loss_sum          : 7.2791     avg(all_tasks_loss_sum)     : 0.2696 
    all tasks_loss_sum_mean     : 6.6574    avg(all_tasks_loss_sum_mean): 0.2466

 + Validation Loop Start - batch_idx:28  eval_iters: 33  t_validation: 33 

 + Validation Loop - batch_idx:28  eval_iters: 33
    loss[task1]     : 0.3173    sum(err)               : 7.5963    sum(err)/batch_id      : 0.2713 
    loss_mean[task1]: 0.2901    sum(err_mean)          : 6.9474    avg(err_mean)          : 0.2481
    self.metrics[task_key][yc_wghts_sum] 140.0    task_weights[task_key] : 3909.0  

 + Validation Loop end - batch_idx:2

IndexError: list index out of range

In [95]:
# environ.display_trained_policy(ns.current_epoch,out=[sys.stdout])
# environ.num_tasks
# print(environ.get_policy_prob().shape)
print(environ.val_data['task1'].keys())
print(environ.val_data['task1']['yc_ind'][0][:40])
print(environ.val_data['task1']['yc_ind'][1][:40])
print(environ.val_data['task1']['yc_data'][:40])
print(environ.val_data['task1']['yc_hat'][:40])


dict_keys(['yc_ind', 'yc_data', 'yc_hat', 'yc_aggr_weights'])
[ 0  1  2  3  4  5  6  7  8  9 10 11 11 12 13 14 15 16 17 18 19 20 20 21 22 23 23 24 25 26 27 28 29 30 31 32 33 34 35 36]
[50 28  6 50 62 28 50 50  6 50 50 53 76 50  0 50 50  6  0  6 50 53 76  0 10 45 52 28 10 50 28 50 71 55  0 71  6 50 38 38]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[ 5.70437     4.990391    4.3287272   5.445829    2.4678407   5.8073773   6.275769    6.7793107   2.1642404   5.247794    5.058744   -2.703812
  2.0684931   5.310354    5.5971427   6.154443    5.8311005   0.01745471  5.625531    3.347758    5.379189   -3.6483507   1.9944686  12.822014
  3.6128433   1.616868    1.422966    4.418747    3.780817    6.8150287   5.8084474   6.3284893   3.542095   -4.6277356  14.590454    2.8610904
  1.4714723   5.6356153   3.6492748   5.145975  ]


In [84]:
print(environ.val_metrics['task1'])
print((environ.val_data['task1']['yc_data']).sum())
print(len(environ.val_data['task1']['yc_ind'][1]))
print(len(environ.val_data['task1']['yc_data']))
print(len(environ.val_data['task1']['yc_hat']))

{'classification': {'_type': 'table-file', 'path': 'media/table/classification_303_06f949eef1c3e2770223.table.json', 'sha256': '06f949eef1c3e2770223ac877bb7bbf4138285b078998733776f70df690c26b7', 'size': 5492, 'artifact_path': 'wandb-client-artifact://it72gol71w4zwp58zxle9snz98k2ywxtf4qndvd2bxwxzkjcfdkssn3nfc1g1uj7sxv1my7vndse2cmmugf9qywm4gq9go3d9t22hhx8fqprfjbca0854vzrjelq6rz9/classification.table.json', '_latest_artifact_path': 'wandb-client-artifact://xfq7c1c8syngcvo0o8w9th5qieyu6fgvajvmb5ra9ug3svw3c08yd17k48o0hazadypv7zncp071os8qeenbvy9ntm0qlwmmtb7c7ittqt0dqbydnk4dt19unzon6uto:latest/classification.table.json', 'ncols': 9, 'nrows': 100}, 'classification_agg': {'roc_auc_score': nan, 'auc_pr': nan, 'avg_prec_score': nan, 'f1_max': nan, 'p_f1_max': nan, 'kappa': nan, 'kappa_max': nan, 'p_kappa_max': nan, 'bceloss': nan, 'sc_loss': 0.00823010015592516, 'logloss': 6.0260329519753785e-05}}
4170.0
4507
4507
4507


In [85]:
(environ.val_data['task1']['yc_data'][0] == environ.val_data['task1']['yc_data']).all()

False

In [104]:
from utils.sparsechem_utils import compute_metrics, aggregate_results
import pandas
cc = compute_metrics(cols   = environ.val_data['task1']['yc_ind'][1], 
                     y_true = environ.val_data['task1']['yc_data'], 
                     y_score= environ.val_data['task1']['yc_hat'] ,
                     num_tasks=100)


 [task is       task  y_true    y_score
14       0     1.0   5.597143
18       0     1.0   5.625531
23       0     1.0  12.822014
34       0     1.0  14.590454
48       0     1.0   6.488068
...    ...     ...        ...
4470     0     1.0  10.922712
4473     0     1.0  13.666399
4475     0     1.0  12.766981
4483     0     1.0  12.470244
4502     0     1.0  13.195760

[318 rows x 3 columns]]
 [task is       task  y_true   y_score
893      3     1.0  5.705761
1315     3     1.0  6.690732
1382     3     1.0  7.397319
2311     3     1.0  9.090773
2364     3     1.0  4.605497
2441     3     1.0  6.213061
2458     3     1.0  6.602612
2548     3     1.0  6.541490
2625     3     1.0  6.570337
2952     3     1.0  8.752759
3413     3     1.0  5.917402
3463     3     1.0  5.819000
3705     3     1.0  4.716419
3730     3     1.0  7.974041
3768     3     1.0  9.193141
4284     3     1.0  8.716344]
 [task is       task  y_true   y_score
2        6     1.0  4.328727
8        6     1.0  2.164240
17  

/home/kbardool/miniconda3/envs/pyt-gpu/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:990: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  warnings.warn(
/home/kbardool/miniconda3/envs/pyt-gpu/lib/python3.9/site-packages/sklearn/metrics/_classification.py:658: RuntimeWarning: invalid value encountered in true_divide
  k = np.sum(w_mat * confusion) / np.sum(w_mat * expected)
/home/kbardool/miniconda3/envs/pyt-gpu/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:990: UndefinedMetricWarning: No negative samples in y_true, false positive value should be meaningless
  warnings.warn(
/home/kbardool/miniconda3/envs/pyt-gpu/lib/python3.9/site-packages/sklearn/metrics/_classification.py:658: RuntimeWarning: invalid value encountered in true_divide
  k = np.sum(w_mat * confusion) / np.sum(w_mat * expected)
/home/kbardool/miniconda3/envs/pyt-gpu/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:990: UndefinedMetricWarning: 

 [task is      task  y_true   y_score
373    99     0.0 -2.167241
712    99     0.0 -2.095917]


/home/kbardool/miniconda3/envs/pyt-gpu/lib/python3.9/site-packages/sklearn/metrics/_classification.py:658: RuntimeWarning: invalid value encountered in true_divide
  k = np.sum(w_mat * confusion) / np.sum(w_mat * expected)
/home/kbardool/miniconda3/envs/pyt-gpu/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:999: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/home/kbardool/miniconda3/envs/pyt-gpu/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:864: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
/home/kbardool/miniconda3/envs/pyt-gpu/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:864: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
/home/kbardool/miniconda3/envs/pyt-gpu/lib/python3.9/site-packages/sklearn/metrics/_classification.py:658: RuntimeWarning: invalid value encountered in true_divide
  k = np.sum(w_mat * confusion) / n

In [88]:
 df   = pd.DataFrame({"task"   : environ.val_data['task1']['yc_ind'][1], 
                      "y_true" : environ.val_data['task1']['yc_data'],  
                      "y_score": environ.val_data['task1']['yc_hat']})

In [94]:
for task, frame in df.groupby("task", sort=True):
    print(f" task {task}")
    print(frame.head(10))

 task 0
    task  y_true    y_score
14     0     1.0   5.597143
18     0     1.0   5.625531
23     0     1.0  12.822014
34     0     1.0  14.590454
48     0     1.0   6.488068
62     0     1.0  13.065102
67     0     1.0   3.847331
68     0     1.0   6.939640
69     0     1.0  13.071250
77     0     1.0  13.010993
 task 3
      task  y_true   y_score
893      3     1.0  5.705761
1315     3     1.0  6.690732
1382     3     1.0  7.397319
2311     3     1.0  9.090773
2364     3     1.0  4.605497
2441     3     1.0  6.213061
2458     3     1.0  6.602612
2548     3     1.0  6.541490
2625     3     1.0  6.570337
2952     3     1.0  8.752759
 task 6
    task  y_true   y_score
2      6     1.0  4.328727
8      6     1.0  2.164240
17     6     1.0  0.017455
19     6     1.0  3.347758
36     6     1.0  1.471472
40     6     1.0  3.464773
47     6     1.0  4.258944
49     6     1.0  3.673662
52     6     1.0  0.113874
57     6     1.0 -0.642786
 task 10
     task  y_true   y_score
24     10     1

In [96]:
# df
df.groupby("task", sort=True).count()

,y_true,y_score
task,,
0,318,318
3,16,16
6,656,656
10,401,401
22,16,16
28,569,569
38,392,392
45,66,66
47,36,36


In [103]:
cc

,roc_auc_score,auc_pr,avg_prec_score,f1_max,p_f1_max,kappa,kappa_max,p_kappa_max,bceloss
task,,,,,,,,,
0,NaN,1.0,1.0,1.0,0.962070,NaN,0.0,1.000000,0.001196
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,1.0,1.0,1.0,0.990102,NaN,0.0,0.999963,0.002298
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
95,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,NaN,1.0,1.0,1.0,0.737796,NaN,0.0,0.946335,0.215865
97,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [109]:
print(environ.batch_data['task1']['yc_aggr_weights'])
environ.batch['task1']['aggr_weights']

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0.]


array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [ ]:
c2 = aggregate_results(cc)

In [12]:
dldrs.trainset0.tasks_weights_list

[namespace(training_weight=tensor([1., 1., 1., 1., 1.]),
           aggregation_weight=array([1., 1., 1., 1., 1.]),
           task_type=None,
           censored_weight=tensor([])),
 namespace(training_weight=tensor([1., 1., 1., 1., 1.]),
           aggregation_weight=array([1., 1., 1., 1., 1.]),
           task_type=None,
           censored_weight=tensor([])),
 namespace(training_weight=tensor([1., 1., 1., 1., 1.]),
           aggregation_weight=array([1., 1., 1., 1., 1.]),
           task_type=None,
           censored_weight=tensor([]))]

In [23]:
# warmup_phase(ns,opt, environ, dldrs, epochs = 25)
ns.best_epoch, ns.best_iter, ns.best_value

(0, 0, 0)

In [110]:
# ns.wandb_run.finish()

ns.wandb_run.finish()

# environ.losses

# environ.val_metrics

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train_time,▂▂▂▂▁▄▃▃▅▃▄▃▅▂▄▆▂▃▁▃█▃▃▂▃▂▃▃▂▇▇▅▂█▄▃▅▂▂▄
epoch,152
train_time,4.03822


#### display parms

In [ ]:
print( f" Backbone Initial LR         : {environ.opt['train']['backbone_lr']:4f}      current LR : {environ.optimizers['alphas'].param_groups[0]['lr']} \n"
       f" Tasks    Initial LR         : {environ.opt['train']['task_lr']:4f}      current LR : {environ.optimizers['weights'].param_groups[0]['lr']}    \n"
       f" Policy   Initial LR         : {environ.opt['train']['policy_lr']:4f}      current LR : {environ.optimizers['weights'].param_groups[1]['lr']}  \n")
print( f" Sparsity regularization     : {environ.opt['train']['lambda_sparsity']}\n"
       f" Sharing  regularization     : {environ.opt['train']['lambda_sharing']} \n\n"
       f" Tasks    regularization     : {environ.opt['train']['lambda_tasks']}   \n"
       f" Gumbel Temp                 : {environ.gumbel_temperature:.4f}         \n" #
       f" Gumbel Temp decay           : {environ.opt['train']['decay_temp_freq']}") #

In [ ]:
# environ.opt['train']['policy_lr'] = 0.01
# opt['train']['policy_lr']         = 0.01
# environ.opt['train']['lambda_sparsity'] = 0.1
# environ.opt['train']['lambda_sharing']  = 0.01
# environ.opt['train']['lambda_tasks']    = 1.0
# environ.opt['train']['decay_temp_freq'] = 2
# print(environ.optimizers['alphas'].param_groups)
# print(environ.optimizers['weights'].param_groups)
# print('current lr: ', environ.optimizers['alphas'].param_groups[0]['lr'],)
# print('current lr: ', environ.optimizers['weights'].param_groups[0]['lr'])
# print('current lr: ', environ.optimizers['weights'].param_groups[1]['lr'])

## Weight & Policy Training

### Weight/Policy Training Preparation

In [ ]:
# ns.flag_warmup = True
# num_train_layers = None 
# environ.opt['is_curriculum'] = True
# environ.opt['curriculum_speed'] = 4
# ns.num_train_layers = None

In [ ]:
if ns.flag_warmup:
    print_heading( f"** {timestring()} \n"
                   f"** Training epoch: {ns.current_epoch} iter: {ns.current_iter}   flag: {ns.flag} \n"
                   f"** Set optimizer and scheduler to policy_learning = True (Switch weight optimizer from ADAM to SGD)\n"
                   f"** Switch from Warm Up training to Alternate training Weights & Policy \n"
                   f"** Take checkpoint and block gradient flow through Policy net", verbose=True)
#     environ.define_optimizer(policy_learning=True)
#     environ.define_scheduler(policy_learning=True)
    ns.flag_warmup = False
    ns.flag = 'update_weights'
    environ.fix_alpha()
    environ.free_weights(opt['fix_BN'])

In [ ]:
# ns.training_epochs = 250
# environ.display_trained_policy(ns.current_epoch)
# environ.display_trained_logits(ns.current_epoch)

In [ ]:
print(f"ns.current_epoch           : {ns.current_epoch}")
print(f"ns.training_epochs         : {ns.training_epochs} \n") 
print(f"ns.current_iters           : {ns.current_iter}")  
print(f"Batches in weight epoch    : {ns.stop_iter_w}")
print(f"Batches in policy epoch    : {ns.stop_iter_a}")
print(f"num_train_layers           : {ns.num_train_layers}")
print()
print_loss(environ.val_metrics, title = f"[e] Last ep:{ns.current_epoch}  it:{ns.current_iter}")
print()

print_heading(f" Last Epoch Completed : {ns.current_epoch}       # of epochs to run:  {ns.training_epochs} -->  epochs {ns.current_epoch+1} to {ns.training_epochs + ns.current_epoch}"
              f"\n policy_learning rate : {environ.opt['train']['policy_lr']} "
              f"\n lambda_sparsity      : {environ.opt['train']['lambda_sparsity']}"
              f"\n lambda_sharing       : {environ.opt['train']['lambda_sharing']}"
              f"\n curriculum training  : {opt['is_curriculum']}     cirriculum speed: {opt['curriculum_speed']}     num_training_layers : {ns.num_train_layers}", 
              verbose = True)

### Weight/Policy Training

In [ ]:
# weight_policy_training(ns, opt, environ, dldrs, epochs = 100)
weight_policy_training(ns, opt, environ, dldrs)

In [ ]:
ns.best_epoch, ns.best_iter, ns.best_value

### Close WandB run

In [ ]:
wandb.finish()

In [ ]:
# ns.best_epoch = 0
# from utils.notebook_modules import wrapup_phase
# wrapup_phase(ns, opt, environ)

In [ ]:
# environ.opt['train']['policy_lr']       = 0.002
# environ.opt['train']['lambda_sparsity'] = 0.05
# environ.opt['train']['lambda_sharing']  = 0.01
# environ.opt['train']['lambda_tasks']    = 1.0
# # environ.opt['train']['decay_temp_freq'] = 2

In [ ]:
print( f" Backbone Learning Rate      : {environ.opt['train']['backbone_lr']}\n"
       f" Tasks    Learning Rate      : {environ.opt['train']['task_lr']}\n"
       f" Policy   Learning Rate      : {environ.opt['train']['policy_lr']}\n")

print( f" Sparsity regularization     : {environ.opt['train']['lambda_sparsity']}\n"
       f" Sharing  regularization     : {environ.opt['train']['lambda_sharing']} \n\n"
       f" Tasks    regularization     : {environ.opt['train']['lambda_tasks']}   \n"
       f" Gumbel Temp                 : {environ.gumbel_temperature:.4f}         \n" 
       f" Gumbel Temp decay           : {environ.opt['train']['decay_temp_freq']}\n") 

print( f" current_iters               : {ns.current_iter}   \n"
       f" current_epochs              : {ns.current_epoch}  \n" 
       f" train_total_epochs          : {ns.training_epochs}\n" 
       f" stop_epoch_training         : {ns.stop_epoch_training}")

## Post Training Stuff

In [ ]:
# pp.pprint(environ.losses)
# pp.pprint(environ.val_metrics)
environ.num_layers, environ.networks['mtl-net'].num_layers

In [ ]:
# pp.pprint(environ.val_metrics['total'])

In [ ]:
# print_loss(environ.val_metrics, title = f"[Final] ep:{current_epoch}  it:{current_iter}",)
# environ.display_trained_policy(current_epoch)
# environ.display_trained_logits(current_epoch)
# environ.log_file.flush()

In [ ]:
# model_label   = 'model_train_ep_%d_seed_%04d' % (current_epoch, opt['random_seed'])
# metrics_label = 'metrics_train_ep_%d_seed_%04d.pickle' % (current_epoch, opt['random_seed'])
# environ.save_checkpoint(model_label, current_iter, current_epoch) 
# save_to_pickle(environ.val_metrics, environ.opt['paths']['checkpoint_dir'], metrics_label)
# print_loss(environ.val_metrics, title = f"[Final] ep:{current_epoch}  it:{current_iter}",)
# environ.display_trained_policy(current_epoch,out=[sys.stdout, environ.log_file])
# environ.display_trained_logits(current_epoch)
# environ.log_file.flush()

In [ ]:
# print_loss(current_iter, environ.losses, title = f"[e] Policy training epoch:{current_epoch}    iter:")
# print()
# print_loss(current_iter, trn_losses, title = f"[e] Policy training epoch:{current_epoch}    iter:")
# print()
# print_loss(current_iter, environ.val_metrics, title = f"[e] Policy training epoch:{current_epoch}    iter:")

In [ ]:
# environ.losses
# environ.val_metrics

In [ ]:
# environ.batch_data
# print_metrics_cr(current_epoch, time.time() - start_time, trn_losses, environ.val_metrics, 0, out=[sys.stdout])
# environ.display_parameters()

# with np.printoptions(edgeitems=3, infstr='inf', linewidth=150, nanstr='nan', precision=7, formatter={'float': lambda x: f"{x:12.5e}"}):
#     environ.print_logit_grads('gradients')

# environ_params = environ.get_task_specific_parameters()
# environ_params = environ.get_arch_parameters()
# environ_params = environ.get_backbone_parameters()
# print(environ_params)
# for param in environ_params:
#     print(param.grad.shape, '\n', param.grad)
#     print(param)

In [ ]:
environ.display_trained_logits(ns.current_epoch)
environ.display_trained_policy(ns.current_epoch)

In [ ]:
environ.display_test_sample_policy(ns.current_epoch, hard_sampling = True)
environ.display_train_sample_policy(ns.current_epoch, hard_sampling = True)

In [ ]:
# environ.define_optimizer(policy_learning=True)

In [ ]:
print(environ.optimizers['alphas'])
print(environ.optimizers['weights'])

In [ ]:
print('Policy  initial_lr : ', environ.optimizers['alphas'].param_groups[0]['initial_lr'], 'lr : ',environ.optimizers['alphas'].param_groups[0]['lr'])
print('Weights initial_lr : ', environ.optimizers['weights'].param_groups[0]['initial_lr'], 'lr : ',environ.optimizers['weights'].param_groups[0]['lr'])
print('Weights initial_lr : ', environ.optimizers['weights'].param_groups[1]['initial_lr'], 'lr : ',environ.optimizers['weights'].param_groups[1]['lr'])

In [ ]:
wandb.run is None

In [ ]:
# opt['exp_instance'] = '0218_1358'     
# folder_name=  f"{opt['exp_instance']}_bs{opt['train']['batch_size']:03d}_{opt['train']['decay_lr_rate']:3.2f}_{opt['train']['decay_lr_freq']}"
# print()
# opt['exp_instance'] = datetime.now().strftime("%m%d_%H%M")
# opt['exp_description'] = f"No Alternating Weight/Policy - training all done with both weights and policy"
# folder_name=  f"{opt['exp_instance']}_bs{opt['train']['batch_size']:03d}_{opt['train']['decay_lr_rate']:3.2f}_{opt['train']['decay_lr_freq']}"

In [ ]:
# wandb.finish()

In [ ]:
# 
p = environ.get_current_state(0)

In [ ]:
pp.pprint(p)

### Post Warm-up Training stuff

In [ ]:
pp.pprint(environ.val_metrics)

In [ ]:
environ.networks['mtl-net'].arch_parameters()

In [ ]:
p = environ.get_sample_policy(hard_sampling = False)
print(p)
p = environ.get_policy_prob()
print(p)
p = environ.get_policy_logits()
print(p)

# p = environ.get_current_policy()
# print(p)

In [ ]:
a = softmax([0.0, 1])
print(a)
sampled = np.random.choice((1, 0), p=a)
print(sampled)

In [ ]:
print(environ.optimizers['weights'])
print(environ.schedulers['weights'].get_last_lr())

In [ ]:
print('losses.keys      : ', environ.losses.keys())
print('losses[task]keys : ', environ.losses['task1'].keys())
pp.pprint(environ.losses)

In [ ]:
print( environ.val_metrics.keys())
# pp.pprint(val_metrics)
print(type(environ.val_metrics['aggregated']))
print()
print(type(environ.val_metrics['task1']['classification_agg']))
print()
pp.pprint(environ.val_metrics)

In [ ]:
# import pickle
# with open("val_metrics.pkl", mode= 'wb') as f:
#         pickle.dump(val_metrics, f)
    
# with open('val_metrics.pkl', 'rb') as f:    
#     tst_val_metrics = pickle.load(f)

In [ ]:
# print(environ.input.shape) 
# a = getattr(environ, 'task1_pred')
# yc_data = environ.batch['task1_data']
# print(yc_data.shape)
# yc_ind = environ.batch['task1_ind']
# print(yc_ind.shape)
# yc_hat_all = getattr(environ, 'task1_pred')
# print(yc_hat_all.shape)
# yc_hat  = yc_hat_all[yc_ind[0], yc_ind[1]]
# print(yc_hat_all.shape, yc_hat.shape)

# 
# environ.losses
# loss = {}
# for key in environ.losses.keys():
#     loss[key] = {}
#     for subkey, v in environ.losses[key].items():
#         print(f" key:  {key}   subkey: {subkey} ")
#         if isinstance(v, torch.Tensor):
#             loss[key][subkey] = v.data
#             print(f" Tensor  -  key:  {key}   subkey: {subkey}           value type: {type(v)}  value: {v:.4f}")
#         else:
#             loss[key][subkey] = v
#             print(f" integer -  key:  {key}   subkey: {subkey}           value type: {type(v)}  value: {v:.4f}")
# pp.pprint(tst_val_metrics)             

In [ ]:
# print('metrics.keys: ', environ.metrics.keys())
# print('metrics[task].keys: ', environ.metrics['task1'].keys())
# pp.pprint(environ.metrics['task1'])
# pp.pprint(environ.losses['task1']['total'])

In [ ]:
# title='Iteration'
# for t_id, _ in enumerate(environ.tasks):
#     task_key = f"task{t_id+1}"
# #     print_heading(f"{title}  {current_iter}  {task_key} : {val_metrics[task_key]['classification_agg']}", verbose = True)

#     for key, _  in val_metrics[task_key]['classification_agg'].items():
#         print('%s/%-20s'%(task_key, key), val_metrics[task_key]['classification_agg'][key], current_iter)
#         print(f"{task_key:s}/{key:20s}", val_metrics[task_key]['classification_agg'][key], current_iter)
#         print()
#             # print_current_errors(os.path.join(self.log_dir, 'loss.txt'), current_iter,key, loss[key], time.time() - start_time)

In [ ]:
# environ.print_loss(current_iter, start_time, metrics = val_metrics['loss'], verbose=True)
# print(opt['lambdas'])
# p = (opt['lambdas'][0] * environ.losses['tasks']['task1'])
# print(p)

# environ.print_val_metrics(current_iter, start_time, val_metrics , title='validation', verbose=True)    

In [ ]:
# print(current_iter)
# print_metrics_cr(current_iter, t1 - t0, None, val_metrics , True)
# environ.print_val_metrics(current_iter, start_time, val_metrics, title='validation', verbose = True)

In [ ]:
print(f" val_metric keys               : {val_metrics.keys()}")
print(f" loss keys                     : {val_metrics['loss'].keys()}")
print(f" task1 keys                    : {val_metrics['task1'].keys()}")
print(f" task1 classification keys     : {val_metrics['task1']['classification'].keys()}")
print(f" task1 classification_agg keys : {val_metrics['task1']['classification_agg'].keys()}")
print()
print(f" task1                       : {val_metrics['task1']['classification_agg']['loss']:5f}")
print(f" task2                       : {val_metrics['task2']['classification_agg']['loss']:5f}")
print(f" task3                       : {val_metrics['task3']['classification_agg']['loss']:5f}")
print(f" loss                        : {val_metrics['loss']['total']:5f}")
print(f" train_time                  : {val_metrics['train_time']:2f}")
print(f" epoch                       : {val_metrics['epoch']}")


### Post Weight + Policy Training Stuff 

In [ ]:
environ.networks['mtl-net'].backbone.layer_config

In [ ]:
num_blocks = 6
num_policy_layers = 6
gt =  torch.ones((num_blocks)).long()
gt0 =  torch.zeros((num_blocks)).long()
print(gt)
print(gt0)

loss_weights = ((torch.arange(0, num_policy_layers, 1) + 1).float() / num_policy_layers)
print(loss_weights)

In [ ]:
if environ.opt['diff_sparsity_weights'] and not environ.opt['is_sharing']:
    print(' cond 1')
    ## Assign higher weights to higher layers 
    loss_weights = ((torch.arange(0, num_policy_layers, 1) + 1).float() / num_policy_layers)
    print(f"{task_key} sparsity error:  {2 * (loss_weights[-num_blocks:] * environ.cross_entropy2(logits[-num_blocks:], gt)).mean()})")
    print_dbg(f" loss_weights :  {loss_weights}", verbose = True)
    print_dbg(f" cross_entropy:  {environ.cross_entropy2(logits[-num_blocks:], gt)}  ", verbose = True)
    print_dbg(f" loss[sparsity][{task_key}]: {self.losses['sparsity'][task_key] } ", verbose = True)

else:
    print('\n cond 2')
    print_dbg(f"Compute CrossEntropyLoss between \n Logits   : \n{logits[-num_blocks:]} \n and gt: \n{gt} \n", verbose = True)
    print(f"{task_key} sparsity error:  {environ.cross_entropy_sparsity(logits[-num_blocks:], gt)}")
    
    print('\n cond 2')
    print_dbg(f"Compute CrossEntropyLoss between Logits      : {logits[-1:]}  and gt: {gt[-1]} ", verbose = True)
    print(f"{task_key} sparsity error:  {environ.cross_entropy_sparsity(logits[-1:], gt[-1:])} \n")
    print_dbg(f"Compute CrossEntropyLoss between Logits      : {logits[-1:]}  and gt: {gt0[-1]} ", verbose = True)
    print(f"{task_key} sparsity error:  {environ.cross_entropy_sparsity(logits[-1:], gt0[-1:])} \n")
    
    print('\n cond 3')    
    print_dbg(f"Compute CrossEntropyLoss between Logits   : {logits[0:1]}  and gt: {gt[0:1]} ", verbose = True)
    print(f"{task_key} sparsity error:  {environ.cross_entropy_sparsity(logits[0:1], gt[0:1])} \n")
    print_dbg(f"Compute CrossEntropyLoss between Logits   : {logits[0:1]}  and gt: {gt0[0:1]} ", verbose = True)
    print(f"{task_key} sparsity error:  {environ.cross_entropy_sparsity(logits[0:1], gt0[0:1])} \n")
        
        

In [ ]:
# flag = 'update_w'
# environ.fix_alpha
# environ.free_w(opt['fix_BN'])

flag = 'update_alpha'
environ.fix_weights()
environ.free_alpha()

In [ ]:
environ.networks['mtl-net'].num_layers

In [ ]:
print(f"current_iters         : {current_iter}")  
print(f"current_epochs           : {current_epoch}") 
print(f"train_total_epochs    : {train_total_epochs}") 

train_total_epochs += 5

print(f"current_iters         : {current_iter}")  
print(f"current_epochs           : {current_epoch}") 
print(f"train_total_epochs    : {train_total_epochs}") 

In [ ]:
# print_metrics_cr(current_epoch, time.time() - t0, None, environ.val_metrics , num_prints)      

# num_prints += 1
# t0 = time.time()

# # Take check point
# environ.save_checkpoint('latest', current_iter)
# environ.train()
# #-------------------------------------------------------
# # END validation process
# #-------------------------------------------------------       
# flag = 'update_alpha'
# environ.fix_w()
# environ.free_alpha()

In [ ]:
# dilation = 2
# kernel_size = np.asarray((3, 3))
# upsampled_kernel_size = (kernel_size - 1) * (dilation - 1) + kernel_size
# print(upsampled_kernel_size)

In [ ]:
# environ.optimizers['weights'].param_groups[0]
# for param_group in optimizer.param_groups:
#     return param_group['lr']

In [ ]:
environ.schedulers['weights'].get_last_lr()

In [ ]:
current_state = {}
for k, v in environ.optimizers.items():
    print(f'state dict for {k} = {v}')
    current_state[k] = v.state_dict()
pp.pprint(current_state)

In [ ]:
current_state = {}
for k, v in environ.schedulers.items():
    print(f'state dict for {k} = {v}')
    print(v.state_dict())

### Losses and Metrics

In [ ]:
trn_losses = environ.losses

In [ ]:
print_metrics_cr(current_epoch, time.time() - start_time, trn_losses, environ.val_metrics , num_prints)      

In [ ]:
# print_metrics_cr(current_epoch, time.time() - start_time, trn_losses, environ.val_metrics , num_prints)      

In [ ]:
# pp.pprint(environ.losses)
pp.pprint(trn_losses)

In [ ]:
pp.pprint(environ.val_metrics)

In [ ]:
# environ.opt['train']['Lambda_sharing'] = 0.5
# opt['train']['Lambda_sharing'] = 0.5

# environ.opt['train']['policy_lr'] = 0.001
# opt['train']['policy_lr'] = 0.001

In [ ]:
environ.losses.keys()
pp.pprint(environ.losses)

In [ ]:
tmp = environ.get_loss_dict()
print(tmp.keys())
pp.pprint(tmp)

In [ ]:
print(opt['diff_sparsity_weights'])
print(opt['is_sharing'])
print(opt['diff_sparsity_weights'] and not opt['is_sharing'])
print(environ.opt['train']['Lambda_sharing'])
print(opt['train']['Lambda_sharing'])
print(environ.opt['train']['Lambda_sparsity'])
print(opt['train']['Lambda_sparsity'])
print(environ.opt['train']['policy_lr'])
print(opt['train']['policy_lr'])

### Policy / Logit stuff

In [ ]:
from scipy.special          import softmax

In [ ]:
np.set_printoptions(precision=8,edgeitems=3, infstr='inf', linewidth=150, nanstr='nan')
torch.set_printoptions(precision=8,linewidth=132)

#### `get_task_logits(n)` Get logits for task group n

In [ ]:
task_logits = environ.get_task_logits(1)
print(task_logits)

#### `get_arch_parameters()`: Get last used logits from network

In [ ]:
import torch.optim as optim
arch_parameters      = environ.get_arch_parameters()
print(arch_parameters)

In [ ]:
import torch.optim as optim
arch_parameters      = environ.get_arch_parameters()
print(arch_parameters)

#### `get_policy_logits()`:  Get Policy Logits - returns same as `get_arch_parameters()`

In [ ]:
logs = environ.get_policy_logits()
for i in logs:
    print(i, '\n')
# probs = softmax(logs, axis= -1)
# for i in probs:
#     print(i, '\n')

#### `get_policy_prob()` : Gets the softmax of the logits

In [ ]:
policy_softmaxs = environ.get_policy_prob()
for i in policy_softmaxs:
    print(i, '\n')

#### `get_sample_policy( hard_sampling = False)` : Calls test_sample_policy of network with random choices based on softmax of logits

In [ ]:
policy_softmaxs = environ.get_policy_prob()
policies,logits = environ.get_sample_policy(hard_sampling = False)

for l, p, s in zip(logits, policies, policy_softmaxs) :
    for  l_row, p_row, s_row in zip(l, p, s):
        print( l_row,'\t', p_row, '\t', s_row)
    print('\n')

#### `get_sample_policy( hard_sampling = True)` : Calls test_sample_policy of network using ARGMAX of logits

In [ ]:
policy_softmaxs = environ.get_policy_prob()
hard_policies, logits = environ.get_sample_policy(hard_sampling = True)

for p,l,s in zip(hard_policies, logits, policy_softmaxs) :
    for  p_row, l_row, s_row in zip(p, l, s):
        print( l_row,'\t', p_row, '\t', s_row)
    print('\n')

#### Print

In [ ]:
print(f" Layer    task 1      task 2      task 3")
print(f" -----    ------      ------      ------")
for idx, (l1, l2, l3) in enumerate(zip(hard_policies[0], hard_policies[1], hard_policies[2]),1):
    print(f"   {idx}      {l1}       {l2}       {l3}")
    

    print(f"\n\n where [p1  p2]:  p1: layer is selected    p2: layer is not selected")

In [ ]:
def display_trained_policy(iter):

    policy_softmaxs = environ.get_policy_prob()
    policy_argmaxs = 1-np.argmax(policy_softmaxs, axis = -1)
    print(f"  Trained polcies at iteration: {iter} ")
    print(f"                   task 1                           task 2                         task 3        ")
    print(f" Layer       softmax        select          softmax        select          softmax        select   ")
    print(f" -----    ---------------   ------       ---------------   ------       ---------------   ------   ")
    for idx, (l1,l2,l3,  p1,p2,p3) in enumerate(zip(policy_softmaxs[0], policy_softmaxs[1], policy_softmaxs[2], policy_argmaxs[0], policy_argmaxs[1], policy_argmaxs[2]),1):
        print(f"   {idx}      {l1[0]:.4f}   {l1[1]:.4f}   {p1:4d}    {l2[0]:11.4f}   {l2[1]:.4f}   {p2:4d}    {l3[0]:11.4f}   {l3[1]:.4f}   {p3:4d}")

    print()
# print(f"\n\n where [p1  p2]:  p1: layer is selected    p2: layer is not selected")

In [ ]:
display_trained_policy(5)

In [ ]:
print(f"                        POLICIES (SOFTMAX)                                       task 3          ")
print(f" Layer    task1              task2            task3 softmax         softmax         argmax         softmax         argmax   ")
print(f" -----    -------------     -------------     -------------   ------   ")
for idx, (l1,l2,l3, h1,h2,h3) in enumerate(zip(policy_softmaxs[0], policy_softmaxs[1], policy_softmaxs[2],hard_policies[0], hard_policies[1], hard_policies[2]),1):
    print(f"   {idx}      {l1[0]:.4f} {l1[1]:.4f}     {l2[0]:.4f} {l2[1]:.4f}     {l3[0]:.4f} {l3[1]:.4f}    {h3}")
    
print(f"\n\n where [p1  p2]:  p1: layer is selected    p2: layer is not selected")

In [ ]:
# print(policy_softmaxs[2], np.argmax(1-policy_softmaxs[2], axis = -1))
print(policy_softmaxs, np.argmax(policy_softmaxs, axis = -1))

#### `get_current_logits()` : Calls test_sample_policy of network using ARGMAX of logits

In [ ]:
logits  = (environ.get_current_logits())
for i in logits:
    print(i ,'\n')

#### `get_current_policy()` : Calls test_sample_policy of network using ARGMAX of logits

In [ ]:
pols  = (environ.get_current_policy())

for i in pols:
    print(i ,'\n')

#### `gumbel_softmax()`  

In [ ]:
np.set_printoptions(precision=8,edgeitems=3, infstr='inf', linewidth=150, nanstr='nan', floatmode = 'maxprec_equal')
torch.set_printoptions(precision=8,linewidth=132)

In [ ]:
print(environ.temp)
# tau = environ.temp
tau = 1
for i in range(3): 
    logits_tensor = torch.tensor(logits[0])
    # Sample soft categorical using reparametrization trick:
    gumbel_soft = F.gumbel_softmax(logits_tensor, tau=tau, hard=False).cpu().numpy() 

    # Sample hard categorical using "Straight-through" trick:
    gumbel_hard  = F.gumbel_softmax(logits_tensor, tau=tau, hard=True).cpu().numpy()
    
    for l, gs, gh in zip(lgts, gumbel_soft, gumbel_hard):
        print(f"   {l}   \t {gs}            \t {gh}")
#     print(lgts)
#     print(gumbel_soft)
#     print(gumbel_hard)
    print()

In [ ]:
for lgts in logits:
    logits_tensor = torch.tensor(lgts)
    print(lgts)
    # Sample soft categorical using reparametrization trick:
    gumbel_soft = F.gumbel_softmax(logits_tensor, tau=1, hard=False)
    print(gumbel_soft)

    # Sample hard categorical using "Straight-through" trick:
    gumbel_hard  = F.gumbel_softmax(logits_tensor, tau=1, hard=True)
    print(gumbel_hard)
    print()

In [ ]:
smax = scipy.special.softmax(logs, axis =1)
# smax = np.array( 
# [[0.46973792, 0.530262  ],
#  [0.45025694, 0.549743  ],
#  [0.4443086 , 0.5556915 ],
#  [0.4138397 , 0.58616036],
#  [0.4140113 , 0.5859887 ],
#  [0.42114905, 0.57885087]])

print(smax.shape)
print(smax)
print(smax[0])
print(smax[0].sum())
print(np.random.choice((1,0), p =smax[0]))

In [ ]:
logs = np.array(
[[0.33064184, 0.42053092],
 [0.3532089 , 0.52056104],
 [0.3888512 , 0.5680909 ],
 [0.42039296, 0.694217  ],
 [0.4519742 , 0.73311865],
 [0.48401102, 0.7522658 ]],
)